In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-04-20 05:13:48.458577: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 05:13:48.458646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 05:13:48.461533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-20 05:13:48.479999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 05:13:51.074818: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../ColorectalHistologicalMNIST/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/5000
[INFO]: Processed 2000/5000
[INFO]: Processed 3000/5000
[INFO]: Processed 4000/5000
[INFO]: Processed 5000/5000
(5000, 224, 224, 3)
(5000,)


In [8]:
from tensorflow.keras.applications import  ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
with tf.device('/CPU:0'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
    model = Model(inputs=base_model.input, outputs=base_model.output)
    features = model.predict(data, batch_size=32, verbose=1)

# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-04-20 05:14:07.227904: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38557 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0


157/157 [==============================] - 293s 2s/step
(5000, 100352)


In [9]:
data = features_flatten

In [10]:
num_classes = len(np.unique(labels))

In [11]:
import gc
import os

def calculate_sensitivity_specificity(y_true, y_pred, class_label):
    # Create a binary confusion matrix for the specified class
    true_positive = sum((y_true == class_label) & (y_pred == class_label))
    false_positive = sum((y_true != class_label) & (y_pred == class_label))
    true_negative = sum((y_true != class_label) & (y_pred != class_label))
    false_negative = sum((y_true == class_label) & (y_pred != class_label))

    # Calculate sensitivity and specificity
    sensitivity = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0

    return sensitivity, specificity
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "EfficientNetB0" or name == "ResNet50":
        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = to_categorical(labels[train_index]), to_categorical(labels[test_index])

                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                #train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                #train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])


                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                # print(y_test)
                # print(y_test_encoded)
                # print(y_pred)
                # print(predictions)
                # Calculate evaluation metrics for the current fold
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivities = []
                specificities = []
                for class_label in range(num_classes):
                    sen, spe = calculate_sensitivity_specificity(y_test_encoded, y_pred, class_label)
                    sensitivities.append(sen)
                    specificities.append(spe)
                sensitivity = sum(sensitivities) / num_classes
                specificity = sum(specificities) / num_classes
                auc_roc = roc_auc_score(y_test, predictions,multi_class='ovr')
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, average='macro')
                f1 = f1_score(y_test_encoded, y_pred, average='macro')
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_prob = model.predict_proba(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivities = []
            specificities = []
            for class_label in range(num_classes):
                sen, spe = calculate_sensitivity_specificity(y_test, y_pred, class_label)
                sensitivities.append(sen)
                specificities.append(spe)
            sensitivity = sum(sensitivities) / num_classes
            specificity = sum(specificities) / num_classes
            auc_roc = roc_auc_score(y_test, y_prob,multi_class='ovr')
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [12]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.906
Sensitivity: 0.906073988735279
Specificity: 0.9865736967493182
AUC-ROC: 0.9932705940507862
MCC: 0.892757056656633
Precision: 0.9057499624031883
F1 Score: 0.9052878313698263

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.936
Sensitivity: 0.9358678955453149
Specificity: 0.990853865605049
AUC-ROC: 0.9961347001146164
MCC: 0.9271968881706206
Precision: 0.9369681067650273
F1 Score: 0.9351971796408003

[INFO] Fold 3 / 10 for LR


...

Accuracy: 0.902
Sensitivity: 0.9019457245263697
Specificity: 0.9859957368107582
AUC-ROC: 0.9919023698116034
MCC: 0.8882426280952842
Precision: 0.905745634329743
F1 Score: 0.9030748625204261

[INFO] Fold 9 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.914
Sensitivity: 0.913978494623656
Specificity: 0.9877132900745014
AUC-ROC: 0.9913133434301746
MCC: 0.9018155618476582
Precision: 0.9157129147712424
F1 Score: 0.9145104073335293

[INFO] Fold 10 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.92
Sensitivity: 0.9200908858166923
Specificity: 0.9885720667063729
AUC-ROC: 0.9955805282882577
MCC: 0.9088964000302773
Precision: 0.9201373104217707
F1 Score: 0.9189644772220118


In [13]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.506
Sensitivity: 0.5054723502304147
Specificity: 0.9294072547360062
AUC-ROC: 0.7172526455253839
MCC: 0.4433199123028973
Precision: 0.5856267492625694
F1 Score: 0.49763462943582776

[INFO] Fold 2 / 10 for NB
Accuracy: 0.536
Sensitivity: 0.5351702508960574
Specificity: 0.9337030970815963
AUC-ROC: 0.7352584748678359
MCC: 0.4767941446719825
Precision: 0.5766103595801337
F1 Score: 0.516021955564112

[INFO] Fold 3 / 10 for NB
Accuracy: 0.526
Sensitivity: 0.5254416282642089
Specificity: 0.9322689727594746
AUC-ROC: 0.7296725610545027
MCC: 0.4668751367413319
Precision: 0.6131487256322861
F1 Score: 0.5150283780963147

[INFO] Fold 4 / 10 for NB
Accuracy: 0.504
Sensitivity: 0.503200204813108
Specificity: 0.9291166421115326
AUC-ROC: 0.7160801718788765
MCC: 0.44135482221525785
Precision: 0.5781812663587289
F1 Score: 0.49728701869817005

[INFO] Fold 5 / 10 for NB
Accuracy: 0.518
Sensitivity: 0.5174411162314388
Specificity: 0.9311195835031294
AUC-ROC: 0.724159121

In [14]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.662
Sensitivity: 0.6621863799283154
Specificity: 0.9517439369716727
AUC-ROC: 0.9217628970290399
MCC: 0.6244919214281808
Precision: 0.7436217723548912
F1 Score: 0.6523400869430058

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.662
Sensitivity: 0.6624743983614951
Specificity: 0.9517406716612854
AUC-ROC: 0.9197983285519158
MCC: 0.6242891019097112
Precision: 0.7395606780224172
F1 Score: 0.6451048088065318

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.664
Sensitivity: 0.6642985151049667
Specificity: 0.9520241006029069
AUC-ROC: 0.9063246421865856
MCC: 0.6253695770893394
Precision: 0.7300644519171304
F1 Score: 0.6490362962995551

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.658
Sensitivity: 0.6585061443932412
Specificity: 0.9511725076538875
AUC-ROC: 0.9111774050376313
MCC: 0.6200468310879207
Precision: 0.7445452511302271
F1 Score: 0.6434730491519242

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.654
Sensitivity: 0.6543778801843317
Specificity: 0.9505978130257149
AUC-ROC: 0.91197

In [15]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.816
Sensitivity: 0.8157962109575012
Specificity: 0.9737135983198019
AUC-ROC: 0.9793122980051909
MCC: 0.7898031314283906
Precision: 0.8153036860807742
F1 Score: 0.8152354612211039

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.864
Sensitivity: 0.8638632872503841
Specificity: 0.9805681378849149
AUC-ROC: 0.982728781228626
MCC: 0.845124046612369
Precision: 0.8643005637402188
F1 Score: 0.8622365636952742

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.85
Sensitivity: 0.8499423963133641
Specificity: 0.9785730332382474
AUC-ROC: 0.9800140964703381
MCC: 0.8288624558552654
Precision: 0.8529269592883248
F1 Score: 0.8503732560887474

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.85
Sensitivity: 0.8498463901689708
Specificity: 0.9785756454865573
AUC-ROC: 0.978172843367922
MCC: 0.8288404877588049
Precision: 0.8557695259132847
F1 Score: 0.8518687540252385

[INFO] Fold 5 / 10 for Random Forest
Accuracy: 0.846
Sensitivity: 0.8456541218637993
Spe

In [12]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1, force_col_wise= True )
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Total Bins 14145929
[LightGBM] [Info] Number of data points in the train set: 4500, number of used features: 99036
[LightGBM] [Info] Start training from score -2.080331
[LightGBM] [Info] Start training from score -2.080331
[LightGBM] [Info] Start training from score -2.078553
[LightGBM] [Info] Start training from score -2.080331
[LightGBM] [Info] Start training from score -2.078553
[LightGBM] [Info] Start training from score -2.078553
[LightGBM] [Info] Start training from score -2.080331
[LightGBM] [Info] Start training from score -2.078553
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

In [13]:
# Save results to a CSV file
results_df.to_csv('DS8_Resnet_LGBM.csv', index=False)

In [14]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LGBM,0.886,0.886018,0.983714,0.987839,0.869945,0.888497,0.886495,2796.808594,44270.824344
